[https://www.kaggle.com/xhlulu/shopee-siamese-resnet-50-image-only-submission?select=submission.csv](http://)

used above notebook as refrence. Thanks to [xhlulu](http:www.kaggle.com/xhlulu). 

I have added following major changes.
* Used efficientnet_b3  model with different projecion head.
* Used some data augmentation techniques for trainig  the network.
* Used Groupkfold for splitting the data to avoid any leakage..

In [ ]:
import sys
#sys.path.append('/workspace/pytorch-image-models-master')
sys.path.append('../input/pytorch-image-models/pytorch-image-models-master')


In [ ]:
MEAN = [0.485, 0.456, 0.406]
STD = [0.229, 0.224, 0.225]

In [ ]:
import pandas as pd
import numpy as np 
import random
import cv2 
import os
import torch
from sklearn.model_selection import GroupKFold
from torch import nn
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
from torch.nn import functional as F
from torch.nn import TripletMarginLoss
import timm
from albumentations.pytorch.transforms import ToTensorV2
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
import torch

In [ ]:
device = ('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
device

In [ ]:
train_path = '../input/shopee-generate-data-for-triplet-loss/train_triplets_ids.csv'
image_path = '../input/shopee-product-matching/train_images'
batch_size= 8
img_size = 256
EPOCHS = 10
SEED = 2020
early_stopping = 4

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [ ]:
seed_everything(SEED)

In [ ]:
train = pd.read_csv(train_path)

In [ ]:
# Merge original data and siamese triplet data

df = pd.read_csv("../input/shopee-generate-data-for-triplet-loss/train_triplets_imgs.csv")

cols = [i +"_path" for i in df.columns]

df = df.rename(columns=dict(zip(df.columns,cols)))

train = pd.concat([train, df],axis=1)

In [ ]:
base = pd.read_csv("../input/shopee-product-matching/train.csv")

In [ ]:
train.head()

In [ ]:
train = pd.merge(train ,base[['posting_id', 'label_group']], left_on=['anchor'], right_on=['posting_id'], how='left')

In [ ]:
train.head()

In [ ]:
# train['label_group'].nunique()

In [ ]:
import albumentations
def get_train_transforms():
    return albumentations.Compose(
        [
            albumentations.HorizontalFlip(p=0.56),
            albumentations.VerticalFlip(p=0.59),
            albumentations.Rotate(limit=120, p=0.8),
            albumentations.RandomBrightness(limit=(0.09, 0.6), p=0.5),
#             albumentations.Cutout(num_holes=8, max_h_size=8, max_w_size=8, fill_value=0, always_apply=False, p=0.5),
#             albumentations.ShiftScaleRotate(
#                 shift_limit=0.25, scale_limit=0.1, rotate_limit=0
#             ),
            albumentations.Normalize(
                MEAN, STD, max_pixel_value=255.0, always_apply=True
            ),
        
            albumentations.Resize(img_size, img_size,always_apply=True),
            ToTensorV2(p=1.0),
            
        ]
    )

def get_valid_transforms():

    return albumentations.Compose([
        albumentations.Resize(img_size, img_size,always_apply=True),
        albumentations.Normalize(MEAN, STD, max_pixel_value=255.0, always_apply=True),
        ToTensorV2(p=1.0)
        ]
    )

In [ ]:
class ShopeeDataset(Dataset):
    
    def __init__(self, df, aug):
        self.df = df.values
        self.aug = aug
        
    def __len__(self):
        return len(self.df)
        
    
    def __getitem__(self, index):
        
        data = dict()
        # Read image
 
        # anchor image
        img_1 = self.df[index][3]
        
        # positive image
        img_2 = self.df[index][4]
        
        # negative image
        img_3 = self.df[index][5]



        
        image_1 = cv2.imread(f'{image_path}/{img_1}')
        image_2 = cv2.imread(f'{image_path}/{img_2}')
        image_3 = cv2.imread(f'{image_path}/{img_3}')
        
        
        image_1 = cv2.cvtColor(image_1, cv2.COLOR_BGR2RGB)
        image_2 = cv2.cvtColor(image_2, cv2.COLOR_BGR2RGB)
        image_3 = cv2.cvtColor(image_3, cv2.COLOR_BGR2RGB)
        
        
            
        image_1 = self.aug(image=image_1)['image']
        image_2 = self.aug(image=image_2)['image']
        image_3 = self.aug(image=image_3)['image']
        
#         image_1 = image_1.permute(2,1,0)
#         image_2 = image_2.permute(2,1,0)
        
        
        data['image_1'] = image_1
        data['image_2'] = image_2
        data['image_3'] = image_3
        
        
        
        return data
        
        
    

In [ ]:
class ShopeeModel(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.model = timm.create_model('efficientnet_b3', pretrained=True)
        n_features = self.model.classifier.in_features
        self.model.classifier = nn.Identity()
        self.model.global_pool =  nn.Identity()
        
        
        self.pooling = nn.AdaptiveAvgPool2d(1)
        self.classfier = nn.Sequential(nn.Linear(n_features, 1024),
                                       nn.Dropout(0.48),
                                       nn.ReLU(),
                                       nn.Linear(1024, 512))
        
        
    
    def forward(self, input):
        
        output = self.model(input)
        output = self.pooling(output).view(input.shape[0], -1)
        #print("shape after pooling",output.shape)
        output = self.classfier(output)
        #print("shape after classification layer", output.shape)
        return output
        

In [ ]:
mdoel = ShopeeModel()

In [ ]:
# class ContrastiveLoss(nn.Module):
#     def __init__(self, margin=0.58):
#         super(ContrastiveLoss, self).__init__()
#         self.margin = margin
        
    
#     def forward(self, output1, output2, output3):
        
#         euclied_dist  = F.pairwise_distance(output1, output2)
#         label = 0
#         loss_p = ((1-label)*(torch.pow(euclied_dist,2)))+(label*torch.clamp(self.margin-torch.pow(euclied_dist,2), min=0.0))
#         loss_p = loss_p/2
         
#         euclied_dist  = F.pairwise_distance(output1, output3)
        
#         label = 1
#         loss_n = ((1-label)*(torch.pow(euclied_dist,2)))+(label*torch.clamp(self.margin-torch.pow(euclied_dist,2), min=0.0))
#         loss_n = loss_n/2
        
#         loss = torch.cat([loss_p, loss_n],axis=0)
#         return torch.mean(loss)

In [ ]:
results = dict()

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
def train_fn(train_dataloader, model, opt, scheduler):
    
    lis = list()
    
    for step, batch in enumerate(train_dataloader):
        
        opt.zero_grad()
        
        image_1 = batch['image_1'].to(device)
        image_2 = batch['image_2'].to(device)
        image_3 = batch['image_3'].to(device)
        
        output_1 = model(image_1)
        output_2 = model(image_2)
        output_3 = model(image_3)
        
        #criterion = ContrastiveLoss().to(device)
        criterion = TripletMarginLoss(margin=0.58).to(device)
        loss = criterion(output_1, output_2, output_3)
        
        loss.backward()
        if (step+1)%4 == 0:
            opt.step()
            scheduler.step()
        else:
            if step == len(train_dataloader):
                opt.step()
                scheduler.step()
        
        lis.append(loss.item())
        
    return lis

def valid_fn(valid_dataloader, model):
    
    lis = list()
    
    model.eval()
    
    
    with torch.no_grad():
        embed = torch.Tensor().to(device)
        labels = torch.Tensor().to(device)
        for step, batch in enumerate(valid_dataloader):



            image_1 = batch['image_1'].to(device)
            image_2 = batch['image_2'].to(device)
            image_3 = batch['image_3'].to(device)

            output_1 = model(image_1)
            output_2 = model(image_2)
            output_3 = model(image_3)


            #criterion = ContrastiveLoss().to(device)
            criterion = TripletMarginLoss(margin=0.58).to(device)
            loss = criterion(output_1, output_2, output_3)

            lis.append(loss.item())
            
            
        
    return lis
    
        


def run(fold):
    early_stopping = 4
    
    train1 = train[train['fold']!=fold]
    valid = train[train['fold']==fold]
    
    train1 = train1.reset_index(drop=True)
    valid = valid.reset_index(drop=True)
    
    train_dataset = ShopeeDataset(train1, get_train_transforms())
    train_dataloder = DataLoader(train_dataset, 
              shuffle=True,
              batch_size=batch_size,
              num_workers=4,
              drop_last=False)
    
    
    valid_dataset = ShopeeDataset(valid, get_valid_transforms())
    valid_dataloader = DataLoader(valid_dataset, 
              shuffle=False,
              batch_size=16,
              num_workers=3,
              drop_last=False)
    
    model = ShopeeModel().to(device)
    
    # Freezing layer
    model.model.requires_grad = False
    
    # Making laster layer of convolution as traianble
    for name,parameter in list(model.children())[0].named_parameters():
        if name =='conv_head.weight':
            print(name)
            parameter.requires_grad = True
            
    opt = Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.0001, weight_decay= 0.001)
    scheduler = CosineAnnealingWarmRestarts(opt, T_0=10, verbose=False)
    
    best_loss = 100000
    for epochs in range(EPOCHS):
        loss_ep = train_fn(train_dataloder, model, opt, scheduler)
        print("Train Loss for epoch", (epochs,sum(loss_ep)/len(loss_ep)))
        
        valid_loss_ep = valid_fn(valid_dataloader, model)
        results[epochs] = valid_loss_ep
        print("Valid Loss for epoch", (epochs,sum(valid_loss_ep)/len(valid_loss_ep)))
        
        if sum(valid_loss_ep)/len(valid_loss_ep)<best_loss:
            early_stopping = 4
            torch.save(model.state_dict(), str(sum(valid_loss_ep)/len(valid_loss_ep))+"_"+str(fold)+"_"+ str(epochs)+".pth")
        else:
            early_stopping = early_stopping-1
            
        if early_stopping==0:
            print(".....early_stopping....")
        

In [ ]:
gkf = GroupKFold(n_splits=2)
groups = train['label_group'].tolist()

for ind, (train_index, val_index) in enumerate(gkf.split(train, train['label_group'], groups)):
    train.loc[val_index, 'fold'] = int(ind)
train['fold'] = train['fold'].astype(int)
print(train.groupby(['fold', 'label_group']).size())

In [ ]:
# Due to computaional constraints training only one fold.
for fold in [1,0]:
    run(fold)
    break


In [ ]:
# train.values[0]

In [ ]:
results[1][0]